In [8]:
import pandas as pd
import numpy as npdd

In [26]:
pats = pd.read_csv('C:/Users/ilcarbo/Documents/DATA/backup cagadas/pats_final.csv')

In [35]:
robos = pd.read_csv('C:/Users/ilcarbo/Documents/DATA/backup cagadas/robos_reg.csv')

In [69]:
# me tengo que quedar solo con los robos de autos patentados en 18 y 19
robos['año_pat'] = robos['fecha_inscripcion_inicial'].apply(lambda x: x.split('-')[0])

In [80]:
robos18_19 = robos[(robos['año_pat'] == '2018') | (robos['año_pat'] == '2019')]

In [81]:
# tiro los "anio origen" = 20
robos18_19.drop(robos[robos['automotor_anio_modelo'] == 20].index, axis = 0, inplace= True)

In [100]:
# cambio genero 'No identificado' por 'No aplica'
robos18_19['titular_genero'].replace('No identificado', 'No aplica', inplace= True)

In [90]:
# tiro los uso "no declarado"
pats.drop(pats[pats['automotor_uso_descripcion'] == 'No declarado'].index, axis = 0, inplace= True)

In [110]:
# tiro los años delirantes de patentamientos
pats.drop(pats[pats['titular_anio_nacimiento'] > 2019].index, axis = 0, inplace= True)

In [115]:
# Rehago la limpieza de nacionalidades
pats['titular_pais_nacimiento'] = pats['titular_pais_nacimiento'].map(lambda x: x.upper())
robos18_19['titular_pais_nacimiento'] = robos18_19['titular_pais_nacimiento'].map(lambda x: x.upper())

In [118]:
pats['titular_pais_nacimiento'] = pats['titular_pais_nacimiento'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
robos18_19['titular_pais_nacimiento'] = robos18_19['titular_pais_nacimiento'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [119]:
pats['titular_pais_nacimiento'].replace(to_replace= '.*ARG.*', value= 'ARGENTINA', regex= True, inplace= True)
pats['titular_pais_nacimiento'].replace(to_replace= '.*URU.*', value= 'URUGUAY', regex= True, inplace= True)
pats['titular_pais_nacimiento'].replace(to_replace= '.*CHI.*', value= 'CHILE', regex= True, inplace= True)
pats['titular_pais_nacimiento'].replace(to_replace= '.*EEUU.*', value= 'ESTADOS UNIDOS', regex= True, inplace= True)
# Cambiar "NO APLICA" y "NO IDENTIFICADO" por NaN // REEMPLAZAR POR NO IDENTIFICADO?
pats['titular_pais_nacimiento'] = pats['titular_pais_nacimiento'].replace('NO APLICA', np.nan)
pats['titular_pais_nacimiento'] = pats['titular_pais_nacimiento'].replace('NO IDENTIFICADO', np.nan)

In [120]:
robos18_19['titular_pais_nacimiento'].replace(to_replace= '.*ARG.*', value= 'ARGENTINA', regex= True, inplace= True)
robos18_19['titular_pais_nacimiento'].replace(to_replace= '.*URU.*', value= 'URUGUAY', regex= True, inplace= True)
robos18_19['titular_pais_nacimiento'].replace(to_replace= '.*CHI.*', value= 'CHILE', regex= True, inplace= True)
robos18_19['titular_pais_nacimiento'].replace(to_replace= '.*EEUU.*', value= 'ESTADOS UNIDOS', regex= True, inplace= True)
# Cambiar "NO APLICA" y "NO IDENTIFICADO" por NaN // REEMPLAZAR POR NO IDENTIFICADO?
robos18_19['titular_pais_nacimiento'] = robos18_19['titular_pais_nacimiento'].replace('NO APLICA', np.nan)
robos18_19['titular_pais_nacimiento'] = robos18_19['titular_pais_nacimiento'].replace('NO IDENTIFICADO', np.nan)

In [124]:
# termino de borrar duplicados y nans
robos18_19.drop('año_pat', axis = 1, inplace= True)

In [134]:
pats.shape

(991192, 18)

In [127]:
pats.drop_duplicates(inplace = True)

In [133]:
pats.dropna(inplace = True)

In [136]:
robos18_19.shape

(1113, 18)

In [130]:
robos18_19.drop_duplicates(inplace = True)

In [135]:
robos18_19.dropna(inplace = True)

-----------------------------------

In [137]:
unif = pd.concat([pats, robos18_19], ignore_index = True)

In [138]:
unif.shape

(992305, 18)

In [140]:
# sacar los patentados que fueron robados para que quede solo trámite de robado
#unif = unif[unif.drop(['tramite_tipo', 'tramite_fecha'],axis=1).duplicated(keep = False)].sort_values(by= ['fecha_inscripcion_inicial', 'automotor_marca_descripcion', 'tramite_fecha'])
unif.sort_values(by= ['fecha_inscripcion_inicial', 'automotor_marca_descripcion', 'tramite_fecha'], inplace= True)
unif.drop_duplicates(subset=unif.columns.difference(['tramite_tipo', 'tramite_fecha']), keep= 'last', inplace = True)

In [141]:
unif.shape

(991268, 18)

In [144]:
unif.tramite_tipo.value_counts()

PATENTADO                   990155
DENUNCIA DE ROBO O HURTO      1113
Name: tramite_tipo, dtype: int64

In [145]:
unif.to_csv('C:/Users/ilcarbo/Documents/DATA/backup cagadas/patentados_robados_juntos_sindups.csv', index= False)